In [ ]:
%matplotlib notebook

In [ ]:
import polars as pl
import numpy as np
import pandas as pd 

from sklearn.preprocessing import LabelEncoder

import os

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb

In [ ]:
print(os.listdir("/kaggle/input/"))

In [ ]:
# reading the training data
path = ('../input/home-credit-credit-risk-model-stability/')

In [ ]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for clm in df.columns:
        # last letter of column name will help you determine the type
        if clm[-1] in ("P", "A"):
            df = df.with_columns(pl.col(clm).cast(pl.Float64).alias(clm))

    return df

In [ ]:
def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [ ]:
# checking missing data
def counting_null_values(df: pl.DataFrame) -> pl.DataFrame:
    total = 0
    
    for col in df.columns: 
        total += df.select(pl.col(col)).null_count()
    
    return(total)


In [ ]:
train_basetable = pl.read_csv(path + "csv_files/train/train_base.csv")
train_static = pl.concat(
    [
        pl.read_csv(path + "csv_files/train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv(path + "csv_files/train/train_static_cb_0.csv").pipe(set_table_dtypes)
train_person_1 = pl.read_csv(path + "csv_files/train/train_person_1.csv").pipe(set_table_dtypes) 
train_credit_bureau_b_2 = pl.read_csv(path + "csv_files/train/train_credit_bureau_b_2.csv").pipe(set_table_dtypes) 
applprev_1 = pl.concat(
    [
        pl.read_csv(path + "csv_files/train/train_applprev_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/train/train_applprev_1_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
other_1 = pl.read_csv(path + "csv_files/train/train_other_1.csv").pipe(set_table_dtypes)
tax_registry_a_1 = pl.read_csv(path + "csv_files/train/train_tax_registry_a_1.csv").pipe(set_table_dtypes)
tax_registry_b_1 = pl.read_csv(path + "csv_files/train/train_tax_registry_b_1.csv").pipe(set_table_dtypes)
tax_registry_c_1 = pl.read_csv(path + "csv_files/train/train_tax_registry_c_1.csv").pipe(set_table_dtypes)
credit_bureau_b_1 = pl.read_csv(path + "csv_files/train/train_credit_bureau_b_1.csv").pipe(set_table_dtypes)
deposit_1 = pl.read_csv(path + "csv_files/train/train_deposit_1.csv").pipe(set_table_dtypes)
person_1 = pl.read_csv(path + "csv_files/train/train_person_1.csv").pipe(set_table_dtypes)
debitcard_1 = pl.read_csv(path + "csv_files/train/train_debitcard_1.csv").pipe(set_table_dtypes)
applprev_2 = pl.read_csv(path + "csv_files/train/train_applprev_2.csv").pipe(set_table_dtypes)
person_2 = pl.read_csv(path + "csv_files/train/train_person_2.csv").pipe(set_table_dtypes)

In [ ]:
dataset_names = [train_basetable , train_static, train_static_cb, train_person_1, train_credit_bureau_b_2, applprev_1, 
                other_1, tax_registry_a_1, tax_registry_b_1, tax_registry_c_1, credit_bureau_b_1, deposit_1,
                person_1, debitcard_1, applprev_2, person_2]
dataset_names_str = ['train_basetable' , 'train_static', 'train_static_cb', 'train_person_1',
                     'train_credit_bureau_b_2','applprev_1', 'other_1', 'tax_registry_a_1', 
                     'tax_registry_b_1', 'tax_registry_c_1', 'credit_bureau_b_1',
                     'deposit_1','person_1', 'debitcard_1', 'applprev_2', 'person_2']
null_value_count = []

for dataset in dataset_names:
    null_value_count.append(pl.lit(dataset.pipe(counting_null_values))) 
    
null_dataset_train = pl.DataFrame(
    {
        "Dataset_Names": dataset_names_str,
        "Null_Value_Count": null_value_count
         
    }
)
null_dataset_train

In [ ]:
credit_bureau_a_1 = pl.concat(
    [
        pl.read_csv(path + "csv_files/train/train_credit_bureau_a_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/train/train_credit_bureau_a_1_1.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_1_2.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_1_3.csv").pipe(set_table_dtypes),

    ],
    how="vertical_relaxed",
)

credit_bureau_a_2 = pl.concat(
    [
        pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_0.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_1.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_2.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_3.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_4.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_5.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_6.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_7.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_8.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_9.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/train/train_credit_bureau_a_2_10.csv").pipe(set_table_dtypes),


    ],
    how="vertical_relaxed",
)


# Explore the Train Datasets

In [ ]:
train_basetable.head()

In [ ]:
train_static.head()

In [ ]:
train_static_cb.head()

In [ ]:
train_person_1.head()

In [ ]:
train_credit_bureau_b_2.head()

In [ ]:
applprev_1.head()

In [ ]:
other_1.head()

In [ ]:
tax_registry_a_1.head()

In [ ]:
tax_registry_b_1.head()

In [ ]:
tax_registry_c_1.head()

In [ ]:
credit_bureau_b_1.head()

In [ ]:
deposit_1.head()

In [ ]:
person_1.head()

In [ ]:
debitcard_1.head()

In [ ]:
applprev_2.head()

In [ ]:
person_2.head()

In [ ]:
credit_bureau_a_1.head()

In [ ]:
credit_bureau_a_2.head()

# Data Exploration

In [ ]:
pmtamount_df = tax_registry_c_1.group_by("case_id", maintain_order=True).agg(pl.sum("pmtamount_36A"))
pmtamount_df

In [ ]:
plt.figure(figsize=(15,6))
plt.title("PMT Amount for Each Case Id", fontweight='bold', fontsize=14)
plt.xlabel("Case Id", fontweight='bold', fontsize=12)
plt.ylabel("PMT Amount", fontweight='bold', fontsize=12)

# define data values
x_axis = np.arange(len(pmtamount_df.head(60))) # X-axis points
y_axis = pmtamount_df["pmtamount_36A"].head(60) # Y-axis points

# Width of each bar
bar_width = 0.6

# Spacing between bars
spacing = 0.7
 
plt.bar(x_axis, y_axis, color='b', alpha=0.5, align="edge",width = bar_width)  # Plot the chart
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, pmtamount_df["case_id"].head(60), rotation="vertical")
plt.xlim(x_axis [0] - spacing, x_axis [-1] + bar_width + spacing)
plt.show()

In [ ]:
dpd_df = applprev_1.group_by("actualdpd_943P", maintain_order=True).agg(pl.col("case_id").count())
dpd_df = dpd_df.sort(by= "actualdpd_943P")
dpd_df = dpd_df.filter(pl.col("actualdpd_943P") != 0)
dpd_df

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Count of Case Id where The DPD doesn't Equal to 0", fontweight='bold', fontsize=14)
plt.ylabel("Count of Case Id", fontweight='bold', fontsize=12)
plt.xlabel("DPD", fontweight='bold', fontsize=12)

# define data values
x_axis =  dpd_df["actualdpd_943P"]# X-axis points
y_axis =  dpd_df["case_id"]# Y-axis points


plt.scatter(x_axis, y_axis, color='b', alpha=0.5)  # Plot the chart

plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.title("DPD for Every Case Id", fontweight='bold', fontsize=14)
plt.xlabel("Case Id", fontweight='bold', fontsize=12)
plt.ylabel("DPD", fontweight='bold', fontsize=12)

# define data values
x_axis =  applprev_1["case_id"]# X-axis points
y_axis = applprev_1["actualdpd_943P"] # Y-axis points

plt.scatter(x_axis, y_axis, color='b', alpha=0.5)  # Plot the chart

plt.show()

In [ ]:
cnt_df = train_static.group_by("applicationcnt_361L", maintain_order=True).agg(pl.col("case_id").count())
cnt_df = cnt_df.filter(pl.col("applicationcnt_361L") != 0)
cnt_df

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Applications with the Same Email Address as the Client", fontweight='bold', fontsize=14)
plt.xlabel("Case Id Count", fontweight='bold', fontsize=10)
plt.ylabel("Number of Applications with the Same Emails", fontweight='bold', fontsize=10)

# define data values
x_axis =  np.arange(len(cnt_df["case_id"])) # X-axis points
y_axis =  cnt_df["applicationcnt_361L"]# Y-axis points

# Width of each bar
bar_width = 0.6

# Spacing between bars
spacing = 0.7
 
plt.bar(x_axis, y_axis, color='r', alpha=0.5, align="edge",width = bar_width)  # Plot the chart
plt.xlim(x_axis [0] - spacing, x_axis [-1] + bar_width + spacing)
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, cnt_df["case_id"].sort())
plt.show()

In [ ]:
thirtyd_df = train_static.group_by("applications30d_658L", maintain_order=True).agg(pl.col("case_id").count())
thirtyd_df = thirtyd_df.filter(pl.col("applications30d_658L") != 0)
thirtyd_df

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Applications Made by the Client in the Last 30 Days", fontweight='bold', fontsize=14)
plt.xlabel("Application Number", fontweight='bold', fontsize=12)
plt.ylabel("Case Id Count", fontweight='bold', fontsize=12)

# define data values
x_axis = thirtyd_df["applications30d_658L"] # X-axis points
y_axis = thirtyd_df["case_id"] # Y-axis points


# Create a list of colors based on the categories
colors = ['green' if category <= 7 else ('yellow' if 7 < category < 14 else 'red') for category in x_axis]

# Plot the scatter chart with the defined colors and alpha value
plt.scatter(x_axis, y_axis, color=colors, edgecolor='black',alpha=0.5)

# Show the plot
plt.show()

In [ ]:
test_basetable = pl.read_csv(path + "csv_files/test/test_base.csv")
test_static = pl.concat(
    [
        pl.read_csv(path + "csv_files/test/test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/test/test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/test/test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv(path + "csv_files/test/test_static_cb_0.csv").pipe(set_table_dtypes)
test_person_1 = pl.read_csv(path + "csv_files/test/test_person_1.csv").pipe(set_table_dtypes) 
test_credit_bureau_b_2 = pl.read_csv(path + "csv_files/test/test_credit_bureau_b_2.csv").pipe(set_table_dtypes)
applprev_1_t = pl.concat(
    [
        pl.read_csv(path + "csv_files/test/test_applprev_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/test/test_applprev_1_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
other_1_t = pl.read_csv(path + "csv_files/test/test_other_1.csv").pipe(set_table_dtypes)
tax_registry_a_1_t = pl.read_csv(path + "csv_files/test/test_tax_registry_a_1.csv").pipe(set_table_dtypes)
tax_registry_b_1_t = pl.read_csv(path + "csv_files/test/test_tax_registry_b_1.csv").pipe(set_table_dtypes)
tax_registry_c_1_t = pl.read_csv(path + "csv_files/test/test_tax_registry_c_1.csv").pipe(set_table_dtypes)
credit_bureau_b_1_t = pl.read_csv(path + "csv_files/test/test_credit_bureau_b_1.csv").pipe(set_table_dtypes)
deposit_1_t = pl.read_csv(path + "csv_files/test/test_deposit_1.csv").pipe(set_table_dtypes)
person_1_t= pl.read_csv(path + "csv_files/test/test_person_1.csv").pipe(set_table_dtypes)
debitcard_1_t = pl.read_csv(path + "csv_files/test/test_debitcard_1.csv").pipe(set_table_dtypes)
applprev_2_t = pl.read_csv(path + "csv_files/test/test_applprev_2.csv").pipe(set_table_dtypes)
person_2_t = pl.read_csv(path + "csv_files/test/test_person_2.csv").pipe(set_table_dtypes)

In [ ]:
dataset_names = [test_basetable , test_static, test_static_cb, test_person_1, test_credit_bureau_b_2, applprev_1, 
                other_1, tax_registry_a_1, tax_registry_b_1, tax_registry_c_1, credit_bureau_b_1, deposit_1,
                person_1, debitcard_1, applprev_2, person_2]
dataset_names_str = ['test_basetable' , 'test_static', 'test_static_cb', 'test_person_1',
                     'test_credit_bureau_b_2','applprev_1', 'other_1', 'tax_registry_a_1', 
                     'tax_registry_b_1', 'tax_registry_c_1', 'credit_bureau_b_1',
                     'deposit_1','person_1', 'debitcard_1', 'applprev_2', 'person_2']
null_value_count = []

for dataset in dataset_names:
    null_value_count.append(pl.lit(dataset.pipe(counting_null_values))) 
    
null_dataset_test = pl.DataFrame(
    {
        "Dataset_Names": dataset_names_str,
        "Null_Value_Count": null_value_count
         
    }
)
null_dataset_test

In [ ]:
credit_bureau_a_1_t = pl.concat(
    [
        pl.read_csv(path + "csv_files/test/test_credit_bureau_a_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/test/test_credit_bureau_a_1_1.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/test/test_credit_bureau_a_1_2.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/test/test_credit_bureau_a_1_3.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/test/test_credit_bureau_a_1_4.csv").pipe(set_table_dtypes),

    ],
    how="vertical_relaxed",
)

credit_bureau_a_2_t = pl.concat(
    [
        pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_0.csv").pipe(set_table_dtypes),
        pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_1.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_2.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_3.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_4.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_5.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_6.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_7.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_8.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_9.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_10.csv").pipe(set_table_dtypes),
#         pl.read_csv(path + "csv_files/test/test_credit_bureau_a_2_11.csv").pipe(set_table_dtypes),



   ],
    how="vertical_relaxed",
)


In [ ]:
test_basetable.head()

In [ ]:
test_static.head()

In [ ]:
test_static_cb.head()

In [ ]:
test_person_1.head()

In [ ]:
test_credit_bureau_b_2.head()

In [ ]:
applprev_1_t.head()

In [ ]:
other_1_t.head()

In [ ]:
tax_registry_a_1_t.head() 

In [ ]:
tax_registry_b_1_t.head()

In [ ]:
tax_registry_c_1_t.head()

In [ ]:
credit_bureau_b_1_t.head()

In [ ]:
deposit_1_t.head()

In [ ]:
person_1_t.head()

In [ ]:
debitcard_1_t.head()

In [ ]:
applprev_2_t.head()

In [ ]:
person_2_t.head()

# Feature engineering

In [ ]:
# Dictionary of Polars DataFrames
tables = {'train_basetable':train_basetable, 'train_static': train_static,'train_static_cb':train_static_cb,
          'train_person_1': train_person_1, 'train_credit_bureau_b_2': train_credit_bureau_b_2, 
          'applprev_1':applprev_1, 'other_1':other_1, 'tax_registry_a_1':tax_registry_a_1, 
          'tax_registry_b_1':tax_registry_b_1,'tax_registry_c_1':tax_registry_c_1 ,
          'credit_bureau_b_1':credit_bureau_b_1, 'deposit_1':deposit_1,'person_1':person_1, 
          'debitcard_1':debitcard_1, 'applprev_2':applprev_2, 'person_2':person_2,
          'credit_bureau_a_1':credit_bureau_a_1,'credit_bureau_a_2':credit_bureau_a_2}

# Specify the target column
target_column_1 = "num_group1"
target_column_2 = "num_group2"


# Loop through each table
print("Table contains", target_column_1, "or", target_column_2 ,":","\n")

for table_name, table in tables.items():
    # Check if the target column exists in the current table
    
    if target_column_1 in table.columns or target_column_2 in table.columns:
              
        # If the column exists, print the table name
       
        print(table_name)

In [ ]:
# Dictionary of Polars DataFrames
table = {'train_basetable':train_basetable, 'train_static': train_static,'train_static_cb':train_static_cb,
          'train_person_1': train_person_1, 'train_credit_bureau_b_2': train_credit_bureau_b_2, 
          'applprev_1':applprev_1, 'other_1':other_1, 'tax_registry_a_1':tax_registry_a_1, 
          'tax_registry_b_1':tax_registry_b_1,'tax_registry_c_1':tax_registry_c_1 ,
          'credit_bureau_b_1':credit_bureau_b_1, 'deposit_1':deposit_1,'person_1':person_1, 
          'debitcard_1':debitcard_1, 'applprev_2':applprev_2, 'person_2':person_2,
          'credit_bureau_a_1':credit_bureau_a_1,'credit_bureau_a_2':credit_bureau_a_2}

# Specify the target column
target_column_1 = "num_group1"
target_column_2 = "num_group2"


# Loop through each table
print("Table contains", target_column_1, "and", target_column_2 ,":","\n")

for table_name, table in tables.items():
    # Check if the target column exists in the current table
    
    if target_column_1 in table.columns and target_column_2 in table.columns:
              
        # If the column exists, print the table name
       
        print(table_name)

In [ ]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or 
# also num_group2 column.
train_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

train_person_1_feats_1

In [ ]:
# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

train_person_1_feats_2

In [ ]:
# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

train_credit_bureau_b_2_feats

In [ ]:
# 
applprev_1_feats = applprev_1.group_by("case_id").agg(
    pl.col("annuity_853A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("actualdpd_943P") > 31).max().alias("actualdpd_943P_above31")
)

applprev_1_feats

In [ ]:
# 
train_credit_bureau_b_1_feat = credit_bureau_b_1.group_by("case_id").agg(
    pl.col("dpd_733P").max().alias("dpd_for_terminated_loans")
)

train_credit_bureau_b_1_feat

In [ ]:
# 
train_credit_bureau_a_1_feat = credit_bureau_a_1.group_by("case_id").agg(
    pl.col("dpdmax_139P").max().alias("dpdmax_for_active_contracts")
)

train_credit_bureau_a_1_feat

In [ ]:
# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("A", "M", "P"):
        selected_static_cols.append(col)
print(selected_static_cols)

In [ ]:
train_static.select(selected_static_cols)

In [ ]:
selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("A", "M", "P"):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)

In [ ]:
train_static_cb.select(selected_static_cb_cols)

In [ ]:
# Join all tables together.
data = train_basetable.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_2_feats, how="left", on="case_id"
).join(
    train_credit_bureau_b_1_feat, how="left", on="case_id"
).join(
    train_credit_bureau_a_1_feat, how="left", on="case_id"
).join(
    applprev_1_feats, how="left", on="case_id"
)

data

In [ ]:
test_person_1_feats_1 = test_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

test_person_1_feats_1

In [ ]:
test_person_1_feats_2 = test_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

test_person_1_feats_2

In [ ]:
test_credit_bureau_b_2_feats = test_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

test_credit_bureau_b_2_feats

In [ ]:
# 
test_applprev_1_feats = applprev_1_t.group_by("case_id").agg(
    pl.col("annuity_853A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("actualdpd_943P") > 31).max().alias("actualdpd_943P_above31")
)

test_applprev_1_feats

In [ ]:
# 
train_credit_bureau_b_1_test_feat = credit_bureau_b_1_t.group_by("case_id").agg(
    pl.col("dpd_733P").max().alias("dpd_for_terminated_loans")
)

train_credit_bureau_b_1_test_feat

In [ ]:
# 
train_credit_bureau_a_1_test_feat = credit_bureau_a_1_t.group_by("case_id").agg(
    pl.col("dpdmax_139P").max().alias("dpdmax_for_active_contracts")
)

train_credit_bureau_a_1_test_feat

In [ ]:
data_submission = test_basetable.join(
    test_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    test_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    test_person_1_feats_1, how="left", on="case_id"
).join(
    test_person_1_feats_2, how="left", on="case_id"
).join(
    test_credit_bureau_b_2_feats, how="left", on="case_id"
).join(
    train_credit_bureau_b_1_test_feat, how="left", on="case_id"
).join(
    train_credit_bureau_a_1_test_feat, how="left", on="case_id"
).join(
    test_applprev_1_feats, how="left", on="case_id"
)


data_submission

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
case_ids = data["case_id"].unique().shuffle(seed=1)
case_ids

In [ ]:
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

In [ ]:
cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

print(cols_pred)

In [ ]:
data.select(cols_pred)

In [ ]:
def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

In [ ]:
base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)
    
df

In [ ]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")

# Training LightGBM

In [ ]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 25,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

In [ ]:
from sklearn .metrics import roc_auc_score

In [ ]:
for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred

print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], base_train["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}') 
print(f'The AUC score on the test set is: {roc_auc_score(base_test["target"], base_test["score"])}')  


In [ ]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

stability_score_train = gini_stability(base_train)
stability_score_valid = gini_stability(base_valid)
stability_score_test = gini_stability(base_test)

print(f'The stability score on the train set is: {stability_score_train}') 
print(f'The stability score on the valid set is: {stability_score_valid}') 
print(f'The stability score on the test set is: {stability_score_test}')  


In [ ]:
X_submission = data_submission[cols_pred].to_pandas()
X_submission = convert_strings(X_submission)
categorical_cols = X_train.select_dtypes(include=['category']).columns

for col in categorical_cols:
    train_categories = set(X_train[col].cat.categories)
    submission_categories = set(X_submission[col].cat.categories)
    new_categories = submission_categories - train_categories
    X_submission.loc[X_submission[col].isin(new_categories), col] = "Unknown"
    new_dtype = pd.CategoricalDtype(categories=train_categories, ordered=True)
    X_train[col] = X_train[col].astype(new_dtype)
    X_submission[col] = X_submission[col].astype(new_dtype)

y_submission_pred = gbm.predict(X_submission, num_iteration=gbm.best_iteration)

In [ ]:
submission = pd.DataFrame({
    "case_id": data_submission["case_id"].to_numpy(),
    "score": y_submission_pred
}).set_index('case_id')
submission.to_csv("./submission.csv")

In [ ]:
submission